DS 4200 - Analyzing Netflix Trends Through IMDB Scores and Entertainment Characteristics
Alissa Agnelli, Paulina Acosta, Regina Rabkina

In [1]:
import pandas as pd
import numpy as np 

In [2]:
imdb_df = pd.read_csv('Netflix TV Shows and Movies.csv')
netflix_df = pd.read_csv('netflix_titles.csv', encoding='ISO-8859-1')

In [3]:
imdb_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5283 entries, 0 to 5282
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              5283 non-null   int64  
 1   id                 5283 non-null   object 
 2   title              5283 non-null   object 
 3   type               5283 non-null   object 
 4   description        5278 non-null   object 
 5   release_year       5283 non-null   int64  
 6   age_certification  2998 non-null   object 
 7   runtime            5283 non-null   int64  
 8   imdb_id            5283 non-null   object 
 9   imdb_score         5283 non-null   float64
 10  imdb_votes         5267 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 454.1+ KB


In [4]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8809 entries, 0 to 8808
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       8809 non-null   object 
 1   type          8809 non-null   object 
 2   title         8809 non-null   object 
 3   director      6175 non-null   object 
 4   cast          7984 non-null   object 
 5   country       7978 non-null   object 
 6   date_added    8799 non-null   object 
 7   release_year  8809 non-null   int64  
 8   rating        8805 non-null   object 
 9   duration      8806 non-null   object 
 10  listed_in     8809 non-null   object 
 11  description   8809 non-null   object 
 12  Unnamed: 12   0 non-null      float64
 13  Unnamed: 13   0 non-null      float64
 14  Unnamed: 14   0 non-null      float64
 15  Unnamed: 15   0 non-null      float64
 16  Unnamed: 16   0 non-null      float64
 17  Unnamed: 17   0 non-null      float64
 18  Unnamed: 18   0 non-null    

In [5]:
netflix_df = netflix_df[['title', 'country', 'date_added', 'release_year', 'duration', 'listed_in', 'rating']]

In [6]:
data = pd.merge(imdb_df, netflix_df, on=['title', 'release_year'], how='inner')

In [7]:
data = data.drop(['id', 'imdb_id', 'index', 'description', 'age_certification', 'duration'], axis = 1)

In [13]:
# Merged the data sets, kept columns that could be relevant to analysis
# Now going to make sure the data types are correct

data['date_added'] = pd.to_datetime(data['date_added'], format='%B %d, %Y', errors='coerce')
data['type'] = data['type'].astype('category')
data['imdb_votes'] = data['imdb_votes'].astype('Int64')
data['rating'] = data['rating'].astype('category')
data['country'] = data['country'].astype('category')
data['listed_in'] = data['listed_in'].astype('category')


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2984 entries, 0 to 2983
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         2984 non-null   object        
 1   type          2984 non-null   category      
 2   release_year  2984 non-null   int64         
 3   runtime       2984 non-null   int64         
 4   imdb_score    2984 non-null   float64       
 5   imdb_votes    2979 non-null   Int64         
 6   country       2742 non-null   category      
 7   date_added    2984 non-null   datetime64[ns]
 8   listed_in     2984 non-null   category      
 9   rating        2984 non-null   category      
dtypes: Int64(1), category(4), datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 182.4+ KB
